In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
import scipy as sp
from scipy.optimize import check_grad
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from os import path

from compton import create_observable_set

In [2]:
mpl.rc('text', usetex=True)
mpl.rc('savefig', transparent=False, bbox='tight', pad_inches=0.05, dpi=300, format='png')
mpl.rcParams['figure.dpi'] = 150

In [3]:
obs_file = path.abspath('../data/polarisabilities-coefficient-table-for-all-observables_20191111_jam.csv')
df = pd.read_csv(obs_file, dtype={'observable': str})

In [4]:
df

,omegalab [MeV],thetalab [deg],observable,nucleon,order,is_numerator,A,B1,B2,B3,...,C33,C34,C35,C36,C44,C45,C46,C55,C56,C66
0,5.0,1,crosssection,proton,3,1,23.550624,-0.001971,-0.001971,4.278199e-07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5.0,1,crosssection,proton,3,0,1.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,5.0,5,crosssection,proton,3,1,23.463893,-0.001963,-0.001963,4.271897e-07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,5.0,5,crosssection,proton,3,0,1.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5.0,10,crosssection,proton,3,1,23.195600,-0.001941,-0.001941,4.251508e-07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284747,340.0,170,2Zp,neutron,4,0,1539.171610,22.347716,-22.350937,4.854397e+01,...,1.775510,-3.452411,3.354500,-3.452411,1.775510,-3.452411,3.354500,1.726380,-3.355539,1.726380
284748,340.0,175,2Zp,neutron,4,1,1404.223925,36.547947,-36.547947,2.941353e+01,...,0.006110,0.012220,-0.012198,-0.012198,0.006110,-0.012198,-0.012198,0.006088,0.012177,0.006088
284749,340.0,175,2Zp,neutron,4,0,1535.901865,22.339037,-22.339236,4.810882e+01,...,1.735146,-3.445853,3.421456,-3.445853,1.735146,-3.445853,3.421456,1.722937,-3.421521,1.722937
284750,340.0,180,2Zp,neutron,4,1,1402.767299,36.552870,-36.552870,2.930796e+01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
df_noback = df[df['thetalab [deg]'] != 180]

In [6]:
omega_lab_unique = df['omegalab [MeV]'].unique()
degrees_lab_unique = df['thetalab [deg]'].unique()
observables_unique = df['observable'].unique()
n_omega = len(omega_lab_unique)
n_angle = len(degrees_lab_unique)

In [7]:
from sklearn.utils.extmath import cartesian
X = cartesian([omega_lab_unique, degrees_lab_unique])
X

array([[  5.,   1.],
       [  5.,   5.],
       [  5.,  10.],
       ...,
       [340., 170.],
       [340., 175.],
       [340., 180.]])

In [11]:
sd_exp = 0.05  # Guess from fig A.9 from Martel Thesis
compton_obs = create_observable_set(df=df_noback, cov_exp=sd_exp**2)

In [12]:
compton_obs

{('1X', 'neutron', 3, 'nonlinear'): 1X(3, neutron),
 ('1X',
  'neutron',
  3,
  'linear'): 1X(3, neutron) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1X', 'neutron', 4, 'nonlinear'): 1X(4, neutron),
 ('1X',
  'neutron',
  4,
  'linear'): 1X(4, neutron) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1X', 'proton', 3, 'nonlinear'): 1X(3, proton),
 ('1X',
  'proton',
  3,
  'linear'): 1X(3, proton) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1X', 'proton', 4, 'nonlinear'): 1X(4, proton),
 ('1X',
  'proton',
  4,
  'linear'): 1X(4, proton) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1Xp', 'neutron', 3, 'nonlinear'): 1Xp(3, neutron),
 ('1Xp',
  'neutron',
  3,
  'linear'): 1Xp(3, neutron) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1Xp', 'neutron', 4, 'nonlinear'): 1Xp(4, neutron),
 ('1Xp',
  'neu

In [13]:
df_obs = pd.read_csv('../data/compton_observables.csv', index_col=False)

In [14]:
df_obs

,omegalab [MeV],thetalab [deg],y0,y2,y3,y4,observable,nucleon
0,5.0,1,0.0,0.000000,0.000000,0.000000,Y,proton
1,5.0,5,0.0,0.000000,0.000000,0.000000,Y,proton
2,5.0,10,0.0,0.000000,0.000000,0.000000,Y,proton
3,5.0,15,0.0,0.000000,0.000000,0.000000,Y,proton
4,5.0,20,0.0,0.000000,0.000000,0.000000,Y,proton
...,...,...,...,...,...,...,...,...
71183,340.0,160,0.0,-0.045191,0.035490,0.005951,3,neutron
71184,340.0,165,0.0,-0.026357,0.019896,0.003274,3,neutron
71185,340.0,170,0.0,-0.012017,0.008820,0.001432,3,neutron
71186,340.0,175,0.0,-0.003050,0.002201,0.000354,3,neutron


In [15]:
df_obs_noback = df_obs[df_obs['thetalab [deg]'] != 179]

In [16]:
df_obs_noback

,omegalab [MeV],thetalab [deg],y0,y2,y3,y4,observable,nucleon
0,5.0,1,0.0,0.000000,0.000000,0.000000,Y,proton
1,5.0,5,0.0,0.000000,0.000000,0.000000,Y,proton
2,5.0,10,0.0,0.000000,0.000000,0.000000,Y,proton
3,5.0,15,0.0,0.000000,0.000000,0.000000,Y,proton
4,5.0,20,0.0,0.000000,0.000000,0.000000,Y,proton
...,...,...,...,...,...,...,...,...
71182,340.0,155,0.0,-0.067343,0.055667,0.009564,3,neutron
71183,340.0,160,0.0,-0.045191,0.035490,0.005951,3,neutron
71184,340.0,165,0.0,-0.026357,0.019896,0.003274,3,neutron
71185,340.0,170,0.0,-0.012017,0.008820,0.001432,3,neutron


In [17]:
np.allclose(df_obs['omegalab [MeV]'], df[(df['is_numerator'] == True) & (df['order'] == 4)]['omegalab [MeV]'])

True

In [18]:
np.allclose(df_obs['thetalab [deg]'][:-1], df[(df['is_numerator'] == True) & (df['order'] == 4)]['thetalab [deg]'][:-1])

False

In [19]:
df_obs['thetalab [deg]'].unique()

array([  1,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,
        65,  70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125,
       130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 179])

In [20]:
df[(df['is_numerator'] == True) & (df['order'] == 4)]['thetalab [deg]'].unique()

array([  1,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,
        65,  70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125,
       130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180])

In [23]:
from compton import proton_pol_vec_mean, neutron_pol_vec_mean

for (obs_name, nucleon, order, approx), comp_obs in compton_obs.items():
    pol_vec = proton_pol_vec_mean if nucleon == 'proton' else neutron_pol_vec_mean
    if approx == 'linear':
        pred = comp_obs.prediction_linear(pol_vec)
    else:
        pred = comp_obs.prediction_ratio(pol_vec)
    df_obs_mask = (df_obs_noback['observable'] == obs_name) & (df_obs_noback['nucleon'] == nucleon)
    val = df_obs_noback[df_obs_mask]['y' + str(order)].values
    is_close = np.allclose(pred, val)
    print(obs_name, nucleon, order, approx, is_close)
    if not is_close:
        print(pred)
        print(val)
        print(np.sum(np.abs(pred-val))/pred.size)
        print(np.abs(pred-val).max())
        print(pred.shape)
        print(val.shape)
        break

1X neutron 3 nonlinear True
1X neutron 3 linear True
1X neutron 4 nonlinear True
1X neutron 4 linear True
1X proton 3 nonlinear True
1X proton 3 linear True
1X proton 4 nonlinear True
1X proton 4 linear True
1Xp neutron 3 nonlinear True
1Xp neutron 3 linear True
1Xp neutron 4 nonlinear True
1Xp neutron 4 linear True
1Xp proton 3 nonlinear True
1Xp proton 3 linear True
1Xp proton 4 nonlinear True
1Xp proton 4 linear True
1Z neutron 3 nonlinear True
1Z neutron 3 linear True
1Z neutron 4 nonlinear True
1Z neutron 4 linear True
1Z proton 3 nonlinear True
1Z proton 3 linear True
1Z proton 4 nonlinear True
1Z proton 4 linear True
1Zp neutron 3 nonlinear True
1Zp neutron 3 linear True
1Zp neutron 4 nonlinear True
1Zp neutron 4 linear True
1Zp proton 3 nonlinear True
1Zp proton 3 linear True
1Zp proton 4 nonlinear True
1Zp proton 4 linear True
2X neutron 3 nonlinear True
2X neutron 3 linear True
2X neutron 4 nonlinear True
2X neutron 4 linear True
2X proton 3 nonlinear True
2X proton 3 linear 